<a href="https://colab.research.google.com/github/laxmipriyapadegal2024/LLM-from-Scratch-101/blob/main/LLMs102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Byte Pair Encoding**

In [ ]:
!pip3 install tiktoken

In [ ]:
import tiktoken,importlib
print("tiktoken version: ",importlib.metadata.version("tiktoken"))

tiktoken version:  0.9.0


In [ ]:
tokenizer=tiktoken.get_encoding("gpt2")

In [ ]:
test=("Hey do you know about dark chocolate? <|endoftext|> I"
      " study computerscience in University.")
ids=tokenizer.encode(test,allowed_special={"<|endoftext|>"})
print(ids)

[10814, 466, 345, 760, 546, 3223, 11311, 30, 220, 50256, 314, 2050, 9061, 4234, 287, 2059, 13]


In [ ]:
text=tokenizer.decode(ids)
print(text)

Hey do you know about dark chocolate? <|endoftext|> I study computerscience in University.


# **CREATING INPUT-TARGET PAIRS**

In [ ]:
with open("the-verdict.txt","r",encoding="utf-8") as f:
  raw_text=f.read()
enc_text=tokenizer.encode(raw_text)
print(len(enc_text))

5145


In [ ]:
import torch

In [ ]:
from torch.utils.data import Dataset, DataLoader
class GPTDatasetV1(Dataset):
  def __init__(self,txt,tokenizer,max_length,stride):
    self.input_ids=[]
    self.output_ids=[]

    token_ids=tokenizer.encode(txt,allowed_special={"<|endoftext|>"})

    for i in range(0,len(token_ids)-max_length,stride):
      input_chunk=token_ids[i:i+max_length]
      output_chunk=token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.output_ids.append(torch.tensor(output_chunk))

  def __len__(self):
    return len(self.input_ids)
  def __getitem__(self,idx):
    return self.input_ids[idx],self.output_ids[idx]

In [ ]:
def create_dataloader_v1(txt,batch_size=4,max_length=256,stride=128,
                         shuffle=True,drop_last=True,num_workers=0):
  tokenizer=tiktoken.get_encoding("gpt2")
  dataset=GPTDatasetV1(txt,tokenizer,max_length,stride)
  dataloader=DataLoader(dataset,
                        batch_size=batch_size,
                        shuffle=shuffle,
                        drop_last=drop_last,
                        num_workers=num_workers)
  return dataloader

In [ ]:
import torch
print("PyTorch version:",torch.__version__)
dataloader=create_dataloader_v1(raw_text,batch_size=128,max_length=4,
                                stride=4,shuffle=False)

data_iter=iter(dataloader)
first_batch=next(data_iter)
print(first_batch)

PyTorch version: 2.6.0+cu124
[tensor([[   40,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026],
        [15632,   438,  2016,   257],
        [  922,  5891,  1576,   438],
        [  568,   340,   373,   645],
        [ 1049,  5975,   284,   502],
        [  284,  3285,   326,    11],
        [  287,   262,  6001,   286],
        [  465, 13476,    11,   339],
        [  550,  5710,   465, 12036],
        [   11,  6405,   257,  5527],
        [27075,    11,   290,  4920],
        [ 2241,   287,   257,  4489],
        [   64,   319,   262, 34686],
        [41976,    13,   357, 10915],
        [  314,  2138,  1807,   340],
        [  561,   423,   587, 10598],
        [  393, 28537,  2014,   198],
        [  198,     1,   464,  6001],
        [  286,   465, 13476,     1],
        [  438,  5562,   373,   644],
        [  262,  1466,  1444,   340],
        [   13,   314,   460,  3285],
        [ 9074,    13, 46606,   536],
        [ 5469,   43

# **Vector/Token Embeddings**

## **Simulation**

In [ ]:
!pip install gensim
!pip uninstall numpy -y
!pip install numpy
!pip uninstall gensim -y
!pip install gensim --upgrade

  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.3 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.5
    Uninstalling numpy-2.2.5:
      Successfully uninstalled numpy-2.2.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.13.1 which is incompatible.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
  Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (62 kB)
Using cached numpy-2.2.5-cp311-cp311-manylinux_2_17_x86_64.ma

In [ ]:
import gensim.downloader as api
model = api.load("word2vec-google-news-300")  # download the model and return as object ready for use

In [ ]:
word_vectors=model
print(word_vectors['computer']) #vector dimension is 300

[ 1.07421875e-01 -2.01171875e-01  1.23046875e-01  2.11914062e-01
 -9.13085938e-02  2.16796875e-01 -1.31835938e-01  8.30078125e-02
  2.02148438e-01  4.78515625e-02  3.66210938e-02 -2.45361328e-02
  2.39257812e-02 -1.60156250e-01 -2.61230469e-02  9.71679688e-02
 -6.34765625e-02  1.84570312e-01  1.70898438e-01 -1.63085938e-01
 -1.09375000e-01  1.49414062e-01 -4.65393066e-04  9.61914062e-02
  1.68945312e-01  2.60925293e-03  8.93554688e-02  6.49414062e-02
  3.56445312e-02 -6.93359375e-02 -1.46484375e-01 -1.21093750e-01
 -2.27539062e-01  2.45361328e-02 -1.24511719e-01 -3.18359375e-01
 -2.20703125e-01  1.30859375e-01  3.66210938e-02 -3.63769531e-02
 -1.13281250e-01  1.95312500e-01  9.76562500e-02  1.26953125e-01
  6.59179688e-02  6.93359375e-02  1.02539062e-02  1.75781250e-01
 -1.68945312e-01  1.21307373e-03 -2.98828125e-01 -1.15234375e-01
  5.66406250e-02 -1.77734375e-01 -2.08984375e-01  1.76757812e-01
  2.38037109e-02 -2.57812500e-01 -4.46777344e-02  1.88476562e-01
  5.51757812e-02  5.02929

In [ ]:
print(word_vectors.most_similar(positive=['king', 'woman'], negative=['man'], topn=10))

[('queen', 0.7118193507194519), ('monarch', 0.6189674139022827), ('princess', 0.5902431011199951), ('crown_prince', 0.5499460697174072), ('prince', 0.5377321839332581), ('kings', 0.5236844420433044), ('Queen_Consort', 0.5235945582389832), ('queens', 0.5181134343147278), ('sultan', 0.5098593831062317), ('monarchy', 0.5087411999702454)]


## **Let's make Token Embeddings**

In [ ]:
import torch
input_ids = torch.tensor([2, 3, 5, 1])
vocab_size = 6
output_dim = 3

torch.manual_seed(123)
embedding_layer = torch.nn.Embedding(vocab_size, output_dim)

In [ ]:
print(embedding_layer.weight)

Parameter containing:
tensor([[ 0.3374, -0.1778, -0.1690],
        [ 0.9178,  1.5810,  1.3010],
        [ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-1.1589,  0.3255, -0.6315],
        [-2.8400, -0.7849, -1.4096]], requires_grad=True)


In [ ]:
print(embedding_layer(torch.tensor([3])))

tensor([[-0.4015,  0.9666, -1.1481]], grad_fn=<EmbeddingBackward0>)


In [ ]:
print(embedding_layer(input_ids))

tensor([[ 1.2753, -0.2010, -0.1606],
        [-0.4015,  0.9666, -1.1481],
        [-2.8400, -0.7849, -1.4096],
        [ 0.9178,  1.5810,  1.3010]], grad_fn=<EmbeddingBackward0>)
